# Demonstration of pyQ3 python wrapper for DEW-ENKI-EQ3 aqueous speciation calculations - TUTORIAL 3

First import the required packages:

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyQ3

## Calculate fluid composition and speciation for a fluid in equilibrium with a rock

More realistic calculations involve equilibrium with minerals. Every element in the system that is not directly specified by molality must be specified by equilibrium with a mineral or solid solution. fO2 and pH are set to a default value or by charge balance unless directly specified.

### Quartz solubility in brine

Create an instance of the default system

In [2]:
dew_system = pyQ3.System()

Set up the elements of the fluid not constrained by mineral equilibria:

In [3]:
molalities = {'NA+':0.05,
              'CL-':0.05,
              }
T = 300.0 + 273.15 # in K
P = 10000.0 # in bar

Tell EQ3 to set the Si content of the solution by equilibrium with Quartz:

In [4]:
mineral_eq = {'H4SIO4(AQ)':'QUARTZ_ALPHA'}

Create the fluid:

In [5]:
dew_fluid = pyQ3.Fluid(dew_system,T,P,molalities=molalities,mineral_eq=mineral_eq)

First we can check that quartz is the stable mineral phase at these conditions:

In [6]:
dew_fluid.mineral_saturation

,mineral,log_q_k,aff,state
0,QUARTZ_ALPHA,0.000,0.000,satd
1,QUARTZ_BETA,-0.120,-0.426,satd
2,COESITE,-0.189,-0.669,NaN
3,HALITE,-4.836,-17.109,NaN
4,CRISTO_ALPHA,-0.435,-1.538,NaN
5,CRISTO_BETA,-0.558,-1.974,NaN


Neither coesite, nor either of the cristobalite polymorphs are stable. The calculation reports that beta-quartz is stable, but this is because the affinity is very close to zero.

In [7]:
dew_fluid.elemental_comp

,ppm,molality
element,,
O,889682.1341,55.607219
H,111991.5253,111.113727
SI,707.1259,0.025178
CL,1772.6500,0.050000
NA,1149.4885,0.050000


But in what species is the Si dissolved?

In [8]:
dew_fluid.aqueous_species

,species,molality,log_g,activity
0,NA+,4.956004e-02,-0.0972,3.962000e-02
1,CL-,4.955850e-02,-0.0972,3.962000e-02
2,H4SIO4(AQ),2.068712e-02,0.0000,2.069000e-02
3,H6SI2O7(AQ),2.228948e-03,0.0000,2.229000e-03
4,NACL(AQ),4.388313e-04,0.0000,4.388000e-04
5,H+,8.550955e-05,-0.0972,6.837000e-05
6,OH-,5.446752e-05,-0.0972,4.355000e-05
7,H3SIO4-,3.258649e-05,-0.0972,2.605000e-05
8,HCL(AQ),2.668582e-06,0.0000,2.669000e-06
9,NAOH(AQ),1.124131e-06,0.0000,1.124000e-06


And what's the pH?

In [9]:
dew_fluid.pH

4.1652

### A fluid in equilibrium with Fayalite-Magnetite-Quartz fO2 buffer

In this example we will allow the fO2 of the fluid to be set by equilibrium with a mineral assemblage. The system involves both Fe and Si, so both of these must also be set:

In [12]:
mineral_eq = {'H4SIO4(AQ)':'QUARTZ_ALPHA',
              'FE+2':'FAYALITE',
              'O2':'MAGNETITE'}
T = 300.0 + 273.15 # in K
P = 10000.0 # in bar

This time we won't add any NaCl.

In [13]:
dew_fluid = pyQ3.Fluid(dew_system, T, P, mineral_eq = mineral_eq)

Hopefully the code has correctly identified that the fO2 is being set by mineral equilibria, and so it hasn't used the abitrary value, let's check:

In [14]:
dew_fluid.fO2

-34.127

Let's check if any other minerals are saturated:

In [15]:
dew_fluid.mineral_saturation

,mineral,log_q_k,aff,state
0,MAGNETITE,0.000,0.000,satd
1,HEMATITE,-0.708,-2.505,NaN
2,FAYALITE,0.000,0.000,satd
3,FERROSILITE,0.052,0.182,satd
4,QUARTZ_ALPHA,0.000,0.000,satd
5,QUARTZ_BETA,-0.120,-0.426,satd
6,COESITE,-0.189,-0.669,NaN
7,IRON_ALPHA,-4.528,-16.020,NaN
8,IRON_GAMMA,-4.785,-16.928,NaN
9,CRISTO_ALPHA,-0.435,-1.538,NaN


Ferrosilite is saturated. Perhaps this isn't a surprise that the FMQ buffer isn't stable everywhere, but is instead a useful reference point...

### Fluids in equilibrium with solid solutions

We can see what the chemistry of a fluid in equilibrium with a fictional harzburgite. First let's see what the default solid solutions built into the module are:

In [16]:
pyQ3.defaultsystem.solid_solutions

{'KFELDSPAR(SS)': ['KFELDSPAR', 'ALBITE'],
 'PLAGIOCLASE(SS)': ['ALBITE', 'ANORTHITE'],
 'ORTHOPYROXENE(SS)': ['ENSTATITE_OR', 'FERROSILITE'],
 'OLIVINE(SS)': ['FORSTERITE', 'FAYALITE'],
 'BIOTITE(SS)': ['PHLOGOPITE', 'ANNITE'],
 'GARNET(SS)': ['PYROPE', 'ALMANDINE', 'GROSSULAR'],
 'CLINOPYROXENE(SS)': ['DIOPSIDE', 'JADEITE', 'HEDENBERGITE'],
 'CPX_SUBCALCIC(SS)': ['DIOPSIDE', 'HEDENBERGITE', 'ENSTATITE_CL'],
 'CA_AMPHIBOLE(SS)': ['TREMOLITE', 'FERROTREMOLITE'],
 'CHLORITE(SS)': ['CLINOCHLORE', 'CHAMOSITE'],
 'CALCITE(SS)': ['CALCITE', 'MAGNESITE', 'SIDERITE'],
 'APATITE(SS)': ['HYDROXYAPATITE', 'FLUORAPATITE', 'CHLORAPATITE']}

Similarly to the pure mineral equilibria, we set the species by equilibrium in the mineral_eq variable, but this time we use a list containing the name of the solid solution, and the name of the endmember.

In [17]:
mineral_eq = {'H4SIO4(AQ)': ['ORTHOPYROXENE(SS)','ENSTATITE_OR'],
              'FE+2': ['OLIVINE(SS)','FAYALITE'],
              'MG+2': ['OLIVINE(SS)','FORSTERITE'],
              'O2':'MAGNETITE'}

We must also specify the solid solution composition of the rock, using a dictionary of dictionaries:

In [18]:
solid_solutions = {'OLIVINE(SS)':{'FORSTERITE':0.9,
                                  'FAYALITE': 0.1},
                   'ORTHOPYROXENE(SS)':{'ENSTATITE_OR':0.8,
                                        'FERROSILITE':0.2}}

Let's generate the fluid.

In [19]:
T = 300.0 + 273.15 # in K
P = 10000.0 # in bar
dew_fluid = pyQ3.Fluid(dew_system,T,P,mineral_eq=mineral_eq, solid_solutions=solid_solutions)

/Users/simonmatthews/opt/anaconda3/lib/python3.7/site-packages/pyQ3/output.py:150: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  ss_info = pd.Series()


How much Mg, Fe, and Si are dissolved in the fluid?

In [20]:
dew_fluid.elemental_comp

,ppm,molality
element,,
O,888303.0062,55.521020
H,111906.9191,111.029784
SI,78.7277,0.002803
FE,0.1239,0.000002
MG,15.0588,0.000620


And what species are present in the fluid?

In [21]:
dew_fluid.aqueous_species

,species,molality,log_g,activity
0,H4SIO4(AQ),2.645681e-03,0.0000,2.646000e-03
1,OH-,1.024263e-03,-0.0213,9.753000e-04
2,MG+2,4.769492e-04,-0.0850,3.921000e-04
3,MG(OH)+,1.364443e-04,-0.0213,1.299000e-04
4,H2(AQ),1.000562e-04,0.0000,1.001000e-04
5,H3SIO4-,7.836982e-05,-0.0213,7.463000e-05
6,H6SI2O7(AQ),3.645650e-05,0.0000,3.646000e-05
7,MG(H3SIO4)+,6.181484e-06,-0.0213,5.886000e-06
8,H+,3.205551e-06,-0.0213,3.052000e-06
9,FE(OH)+,1.184361e-06,-0.0213,1.128000e-06


The pH?

In [22]:
dew_fluid.pH

5.5154

Is this a feasible mineral assemblage?

In [23]:
dew_fluid.mineral_saturation

,mineral,log_q_k,aff,state
0,MAGNETITE,0,0,satd
1,HEMATITE,-0.655,-2.316,NaN
2,PERICLASE,-2.364,-8.363,NaN
3,BRUCITE,-0.412,-1.458,NaN
4,FORSTERITE,-0.046,-0.162,satd
5,FAYALITE,-1,-3.538,NaN
6,CHRYSOTILE,2.103,7.440,ssatd
7,ENSTATITE_CL,-0.057,-0.201,satd
8,ENSTATITE_OR,-0.097,-0.343,satd
9,ENSTATITE_PR,-0.27,-0.956,NaN


Well, it looks like a fluid infiltrating the rock will react with the mineral assemblage to form some hydrated phases... This seems likely to happen in reality! This could be a good starting point for an EQ6 reaction path calculation...